In [106]:
import mlflow
import os

# Set URI to your DagsHub repository's MLflow tracking endpoint
mlflow.set_tracking_uri("https://dagshub.com/kumarisudha11122003-spec/datascienceproject.mlflow")

# Set up authentication using your DagsHub username and token (environment variables)
os.environ["MLFLOW_TRACKING_USERNAME"] = "kumarisudha11122003-spec"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "e43ca27d45b6b9e814548f356ba0551865dd7ae5"

In [107]:
import os

In [108]:
%pwd

'f:\\datascienceproject'

In [4]:
os.chdir("../")
%pwd

'f:\\datascienceproject'

In [109]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [110]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml,create_directories,save_json

In [111]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/kumarisudha11122003-spec/datascienceproject.mlflow"


        )
        return model_evaluation_config

In [121]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [122]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        #mlflow.set_registry_uri(self.config.mlflow_uri)
        #tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)



            mlflow.sklearn.log_model(model, "model")
    
            
    
           


In [123]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-11-19 21:08:17,753: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-11-19 21:08:17,909: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-19 21:08:17,984: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-11-19 21:08:18,051: INFO: common: created directory at: artifacts]
[2025-11-19 21:08:18,064: INFO: common: created directory at: artifacts/model_evaluation]
[2025-11-19 21:08:23,853: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2025/11/19 21:08:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run legendary-lynx-185 at: https://dagshub.com/kumarisudha11122003-spec/datascienceproject.mlflow/#/experiments/0/runs/85094caf38f6467594df660fcf35bdb5
🧪 View experiment at: https://dagshub.com/kumarisudha11122003-spec/datascienceproject.mlflow/#/experiments/0


RestException: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}